# Synthesio Automation via Selenium


Author: Zening ('Ze') Duan
Date: 2020-01-22

## 1. Load Packages

In [1]:
#!pip install selenium
import time
import datetime
import re
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from utlis_v5 import *
from datetime import timedelta
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select

import pandas as pd

## 2. Run

## 2.1 Log in

In [2]:
Username = 'XXXX' #Enter your username
Password = 'XXXX' #Enter your password

Filters = {
           'Topics': ['all',
                      'user_defined'
                     ],
           'Publisher categories': ['Twitter',
                                   'Facebook'
                                   ],
           #only support non-default filters: lang and geo for this version. Update the non_default_filters dict if needed
           'Languages': ['English'
                         ,'Chinese'
                        ],
            
           'Countries': {'North America': ['United States'],
                         'Africa': ['Morocco', 'Algeria', 'Tunisia'],
                         'Asia': ['India', 'Japan', 'China']
                        }
          } 

Dashboard = str('XXX')#enter your dashboard name, e.g., "CAMER (Copy)"

 

Begin_date = {'Year': '2020',
             'Month': '06',
             'Day': '17',
              'Hour': '00',
              'Minute': '00'
             }

End_date = {'Year':'2020' ,
             'Month': '06',
             'Day': '18',
            'Hour': '23',
              'Minute': '59'
           }


Time_interval = float(1.0) #in day, change it if needed

Sample = {'mode': 'head',
                   'random'
                  ,
    'fraction': float(1)
}

Data_format = ['XLSX', 'CSV'][1]

In [ ]:
driver = setup(Username, Password, Dashboard, Filters, Data_format, Sample)

Begin_date_obj = date_obj(Begin_date) 
End_date_obj = date_obj(End_date) 

Delta = timedelta(days=Time_interval) 


#Run
Current_time = Begin_date_obj

while Current_time < End_date_obj:
    try:
        timeout = driver.find_element_by_xpath("/html/body/div[490]/div/div/section") #check if the timeout page occurs
        if 'Session timeout' == timeout.text:
            print('Session TimeOut')
        
            if len(driver.window_handles) > 1:
                driver.close()

            driver.switch_to.window(driver.window_handles[0])
            driver.execute_script("window.open('');")
            driver.switch_to.window(driver.window_handles[1])
    
            """
            #driver = webdriver.Firefox(executable_path = '/usr/local/Cellar/python/3.7.6_1/geckodriver')
            #driver.get("https://app.synthesio.com/dashboard/#home/workspace/35944/projects/dashboard")
            #driver.execute_script('''window.open("https://rest.synthesio.com/login/v1/login?login_challenge=064ecd2f3d5e4443b83d8b7e4e89aa23#home/workspace/35944/projects/dashboard", "_blank");''') #login URL      
            """
        
            driver = resetup(driver, Username, Password, Dashboard, Filters, Data_format, Sample)# re-set up other filters
    
    except:
        pass
    
    print(strfdate(Current_time))
    sampling(driver, Current_time, Delta)
    volume_total,volume = dataexport(driver, Data_format, Sample)
    print('volumetotal: ' + str(volume_total) + ', volume: ' + str(volume))

    
    VolumeTablePath = 'xxxxx' # type in path of the volume table
    data = pd.read_csv(VolumeTablePath)
    data['Volume_total'].loc[(data['Year'] == int(Current_time.year)) & (data['Month'] == int(Current_time.month)) & (data['Day'] == int(Current_time.day))] = volume_total
    data['Volume'].loc[(data['Year'] == int(Current_time.year)) & (data['Month'] == int(Current_time.month)) & (data['Day'] == int(Current_time.day))] = volume
    data.to_csv(VolumeTablePath, index = False)
    
    
    tic = time.time()
    #waiting
    time.sleep(300) #change here based on the expected volume per click. e.g., 1000s for 50k (~14 mins baseline)

    #status checking
    count = 0 
    while count < 70: #change here based on the your patience, e.g., 20 mins more
        driver, flag = dataexportcheck(driver)
        if flag == True:
            print('Next round download is ready!')
            break
        else:
            count += 1
            print(count)
            time.sleep(30)

    toc = time.time()
    print('This task costs ' + str(int(toc - tic)/60) + 'min.')

    Current_time += Delta
    

We currently have  5  Dashboards:  ['Market Research', 'MeToo', 'CAMER', 'CAMER (Copy)', 'Immigration']
The location of the dashboard we are seeking for is: Index[ 3 ].
2020/06/17/00:00
The total Volume is  44909 . Data is ready to download.
This task, we downloaded 44909 mentions.


/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


text found
1
text found
2
text found
3
text found
4
text found
5
text found
6
text found
7
text found
8
text found
9
text found
10
text not found!
Next round download is ready!
This task costs 26.75min.
2020/06/17/01:00
The total Volume is  44038 . Data is ready to download.
This task, we downloaded 44038 mentions.


/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


text found
1
text found
2
text found
3
text found
4
text found
5
